## 라이브러리 import

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

## 데이터 로드

In [8]:
X_train, X_test, Y_train, Y_test = np.load('./data/nouns_books_data_max_612_size_68561.npy', allow_pickle=True)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(52547, 612) (5839, 612) (52547, 8) (5839, 8)


## 모델생성

In [9]:
model = Sequential()
model.add(Embedding(input_dim=68561,   # 단어들을 벡터공간에 배치 / 의미상 유사한 단어들은 가까운 지점으로 매핑 / 연산 가능해짐 / 복잡하고 연산량 많음
                    output_dim=350,  # 차원 축소 / 데이터 학습을 위해 차원이 증가하면서 학습데이터 수가 차원의 수보다 적어져 성능이 저하 -> 차원의 저주
                    input_length=612))  # CNN의 input_shape과 같음
model.add(Conv1D(32, kernel_size=9, padding='same', activation='relu'))  # 주변 값과의 연관성을 분석하기 위해 사용
model.add(MaxPool1D(pool_size=1))  # 그냥 Conv 친구라 ㅎㅎ
model.add(GRU(128, activation='tanh'))
model.add(Dropout(0.6))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(8, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 612, 350)          23996350  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 612, 32)           100832    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 612, 32)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               62208     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 128)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)              

## 모델 학습

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_hist = model.fit(X_train, Y_train, batch_size=512, epochs=4, validation_data=(X_test, Y_test))

Epoch 1/4
103/103 [==============================] - 1446s 14s/step - loss: 1.5749 - accuracy: 0.3774 - val_loss: 0.7869 - val_accuracy: 0.7131
Epoch 2/4
103/103 [==============================] - 1437s 14s/step - loss: 0.7454 - accuracy: 0.7465 - val_loss: 0.5171 - val_accuracy: 0.8546
Epoch 3/4
103/103 [==============================] - 1458s 14s/step - loss: 0.4472 - accuracy: 0.8664 - val_loss: 0.4300 - val_accuracy: 0.8952
Epoch 4/4
103/103 [==============================] - 1442s 14s/step - loss: 0.2900 - accuracy: 0.9227 - val_loss: 0.4817 - val_accuracy: 0.9008


## 모델 검증 및 저장

In [11]:
score = model.evaluate(X_test, Y_test)
print(score[1])

183/183 [==============================] - 37s 203ms/step - loss: 0.4817 - accuracy: 0.9008
0.9008392095565796


In [12]:
model.save(f'./models/news_classification_{score[1]:.4f}.h5')